In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler, \
    RobustScaler, Normalizer, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")


# setting number of columns and rows to be shown in output
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 10)

# Loading Data

In [2]:
# using adult data from UCI ML repository - https://archive.ics.uci.edu/ml/datasets/adult
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None)

In [3]:
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
               'marital-status', 'occupation', 'relationship', 'race', 'sex',
               'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
               'salary']

In [4]:
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
                   ...  
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
Length: 15, dtype: object

In [6]:
# import just a few columns
x = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv', usecols=['crim', 'medv'])
x.head()

,crim,medv
0,0.00632,24.0
1,0.02731,21.6
2,0.02729,34.7
3,0.03237,33.4
4,0.06905,36.2


# Descriptive Statistics

In [7]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [8]:
# skew
df.fnlwgt.skew()

1.4469800945789826

In [9]:
# kurtosis
df.fnlwgt.kurt()

6.218810978153801

In [10]:
# correlation for numeric columns
df[["age", "fnlwgt", "education-num", "capital-gain"]].corr()

,age,fnlwgt,education-num,capital-gain
age,1.000000,-0.076646,0.036527,0.077674
fnlwgt,-0.076646,1.000000,-0.043195,0.000432
education-num,0.036527,-0.043195,1.000000,0.122630
capital-gain,0.077674,0.000432,0.122630,1.000000


In [11]:
# value counts for catogorical variables
df.sex.value_counts()

 Male      21790
 Female    10771
Name: sex, dtype: int64

In [12]:
# unique values for catogorical variables
df.sex.unique()

array([' Male', ' Female'], dtype=object)

# Apply function

In [13]:
tmp = pd.DataFrame(np.random.randint(1,100, 80).reshape(8, -1))

# create a new column 'penultimate' which has the second largest value of each row of dataframe
# axis = 1 checks row wide
out = tmp.apply(lambda x: x.sort_values().unique()[-2], axis=1)
tmp['penultimate'] = out
tmp

,0,1,2,3,4,5,6,7,8,9,penultimate
0,50,7,59,23,75,8,12,4,32,78,75
1,64,31,11,62,70,49,54,26,41,88,70
2,27,34,11,52,60,60,43,16,18,36,52
3,80,45,70,86,18,62,49,89,45,43,86
4,80,92,90,9,91,73,48,75,2,36,91
5,63,80,4,6,56,73,24,56,93,99,93
6,64,5,39,75,32,41,95,92,59,64,92
7,49,59,50,22,68,84,65,63,13,53,68


In [14]:
# apply to each cloumn and row
# normalize all columns of df by subtracting the column mean and divide by standard deviation

tmp = pd.DataFrame(np.random.randint(1,100, 80).reshape(8, -1))

# apply default axis = 0, so we calculate mean and std column wide
tmp.apply(lambda x: ((x - x.mean())/x.std()).round(2))

,0,1,2,3,4,5,6,7,8,9
0,-0.93,0.99,1.30,-0.48,-0.13,1.09,0.08,0.73,0.15,1.17
1,-0.53,-0.93,-0.87,0.88,-0.67,0.39,1.82,-0.38,-0.07,-0.92
2,1.44,1.31,1.39,-0.44,0.74,-0.52,-0.15,1.12,1.05,0.72
3,0.90,-1.04,-0.43,1.44,-0.29,-1.35,-1.47,-1.55,1.15,-0.74
4,-0.61,-0.96,0.24,0.24,1.01,0.53,-0.56,-0.53,-1.14,1.14
5,1.08,1.12,-1.00,-1.36,-1.98,1.20,0.72,1.36,-1.08,-1.13
6,-0.18,-0.08,-1.09,0.80,0.47,0.01,-0.75,-0.72,0.99,0.64
7,-1.18,-0.42,0.46,-1.08,0.85,-1.35,0.31,-0.04,-1.05,-0.87


In [15]:
tmp = pd.DataFrame(np.random.randint(1,100, 80).reshape(8, -1), columns=list('pqrstuvwxy'), index=list('abcdefgh'))
tmp

,p,q,r,s,t,u,v,w,x,y
a,47,11,12,10,21,68,29,14,73,5
b,54,35,58,17,19,27,81,26,42,84
c,38,85,57,92,31,37,81,81,89,70
d,94,59,82,4,3,6,61,37,12,11
e,26,18,61,79,86,87,40,43,28,57
f,10,29,27,71,52,19,87,45,42,56
g,49,97,85,25,24,84,77,58,2,69
h,81,12,17,60,14,94,17,39,49,58


In [16]:
# max possible correlation for each column
abs_corrmat = np.abs(df.corr())
max_corr = abs_corrmat.apply(lambda x: sorted(x)[-2]) # -2 because max correlation is 1 with column itself
print('Maximum Correlation possible for each column: ', np.round(max_corr.tolist(), 2))

Maximum Correlation possible for each column:  [0.08 0.08 0.15 0.12 0.08 0.15]


In [17]:
# column name with the highest number of row-wise maximum’s in dataframe
tmp = pd.DataFrame(np.random.randint(1,100, 40).reshape(10, -1))
tmp

,0,1,2,3
0,85,81,73,44
1,74,11,44,71
2,30,44,22,86
3,1,91,49,30
4,94,96,45,54
5,48,74,75,48
6,97,68,97,59
7,22,47,28,70
8,99,66,91,95
9,51,96,29,95


In [18]:
# use idxmax similar to np.argmax
#  df.apply(np.argmax, axis=1).value_counts().index[0]
tmp.idxmax(axis=1).value_counts().index[0]

0

In [19]:
# passing additional arguments to apply lambda function
d = 2
tmp.iloc[:,0].apply(lambda x,d: x+d, args=(d, ))

0     87
1     76
2     32
3      3
4     96
5     50
6     99
7     24
8    101
9     53
Name: 0, dtype: int64

# Iterating over rows

In [20]:
for index, row in df.head().iterrows():
    print(index, row['age'])

0 39
1 50
2 38
3 53
4 28


In [21]:
# create a new column such that, each row contains the row number of nearest row-record by euclidean distance
tmp = pd.DataFrame(np.random.randint(1,100, 40).reshape(10, -1), columns=list('pqrs'), index=list('abcdefghij'))

# Solution
import numpy as np

# init outputs
nearest_rows = []
nearest_distance = []

# iterate rows.
for i, row in tmp.iterrows():
    curr = row
    rest = tmp.drop(i)
    e_dists = {}  # init dict to store euclidean dists for current row.
    # iterate rest of rows for current row
    for j, contestant in rest.iterrows():
        # compute euclidean dist and update e_dists
        e_dists.update({j: round(np.linalg.norm(curr.values - contestant.values))})
    # update nearest row to current row and the distance value
    nearest_rows.append(max(e_dists, key=e_dists.get))
    nearest_distance.append(max(e_dists.values()))

tmp['nearest_row'] = nearest_rows
tmp['dist'] = nearest_distance

tmp

,p,q,r,s,nearest_row,dist
a,96,41,1,51,c,114.0
b,40,16,96,47,a,113.0
c,4,71,56,25,a,114.0
d,69,79,81,25,f,124.0
e,66,98,74,28,f,130.0
f,7,6,19,73,e,130.0
g,82,31,11,73,c,110.0
h,89,21,53,34,c,99.0
i,42,34,46,94,e,99.0
j,43,70,1,62,b,110.0


# Group by

### How groupby works
  
Divide, aggrigate and combine. 
  
![title](images/group_by.jpg)

In [22]:
df.groupby('sex').fnlwgt.mean()

sex
 Female    185746.311206
 Male      191771.449013
Name: fnlwgt, dtype: float64

In [23]:
# as_index = False doesn't create multi index for group by
df.groupby('sex', as_index=False).fnlwgt.mean()

,sex,fnlwgt
0,Female,185746.311206
1,Male,191771.449013


In [24]:
df.groupby('sex').fnlwgt.agg(['mean', 'min', 'max'])

,mean,min,max
sex,,,
Female,185746.311206,12285,1484705
Male,191771.449013,13769,1455435


In [25]:
# use reset_index to get rid of multiindex
df.groupby('sex').fnlwgt.agg(['mean', 'min', 'max']).reset_index()

,sex,mean,min,max
0,Female,185746.311206,12285,1484705
1,Male,191771.449013,13769,1455435


In [26]:
fruits = pd.DataFrame({'fruit': ['apple', 'banana', 'orange'] * 3,
                   'taste': np.random.rand(9),
                   'price': np.random.randint(0, 15, 9)})
fruits

,fruit,taste,price
0,apple,0.460023,4
1,banana,0.976737,12
2,orange,0.759283,6
3,apple,0.715987,7
4,banana,0.230587,2
5,orange,0.526256,3
6,apple,0.591191,4
7,banana,0.307205,6
8,orange,0.330492,9


In [27]:
# use get_group to get group from group by
fruits.groupby('fruit').taste.get_group('banana')

1    0.976737
4    0.230587
7    0.307205
Name: taste, dtype: float64

In [28]:
users = pd.read_table('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user', 
                      sep='|', index_col='user_id')
users.head()

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [29]:
# the male ratio per occupation and sort it from the most to the least
# create a function
def gender_to_numeric(x):
    if x == 'M':
        return 1
    if x == 'F':
        return 0

# apply the function to the gender column and create a new column
users['gender_n'] = users['gender'].apply(gender_to_numeric)


a = users.groupby('occupation').gender_n.sum() / users.occupation.value_counts() * 100 

# sort to the most male 
a.sort_values(ascending = False).head()

doctor        100.000000
engineer       97.014925
technician     96.296296
retired        92.857143
programmer     90.909091
dtype: float64

In [30]:
# min and max age in an occupation by gender
users.groupby('occupation').age.agg(['min', 'max', 'mean']).head()

,min,max,mean
occupation,,,
administrator,21,70,38.746835
artist,19,48,31.392857
doctor,28,64,43.571429
educator,23,63,42.010526
engineer,22,70,36.388060


In [31]:
# reset index to remove multi index after group by
# https://jamesrledoux.com/code/group-by-aggregate-pandas
users.groupby('occupation').age.agg(['min', 'max', 'mean']).reset_index().head()

,occupation,min,max,mean
0,administrator,21,70,38.746835
1,artist,19,48,31.392857
2,doctor,28,64,43.571429
3,educator,23,63,42.010526
4,engineer,22,70,36.388060


In [32]:
# for each occupation present the percentage of women and men

# create a data frame and apply count to gender
# agg({'gender': 'count'}) means on gender column do count agggrigate
gender_ocup = users.groupby(['occupation', 'gender']).agg({'gender': 'count'})

# create a DataFrame and apply count for each occupation
occup_count = users.groupby(['occupation']).agg('count')

# divide the gender_ocup per the occup_count and multiply per 100
occup_gender = gender_ocup.div(occup_count, level = "occupation") * 100

# present all rows from the 'gender column'
occup_gender.loc[: , 'gender'].head(10)

occupation     gender
administrator  F          45.569620
               M          54.430380
artist         F          46.428571
               M          53.571429
doctor         M         100.000000
educator       F          27.368421
               M          72.631579
engineer       F           2.985075
               M          97.014925
entertainment  F          11.111111
Name: gender, dtype: float64

# Sampling

In [33]:
df.sample(n=3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
15359,48,Private,299291,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,44,United-States,<=50K
20129,41,Private,204235,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,>50K
23625,34,Self-emp-not-inc,203488,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,United-States,<=50K


In [34]:
df.shape

(32561, 15)

In [35]:
df.sample(frac=0.1).shape

(3256, 15)

In [36]:
df.sample(frac=0.1).head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
26667,59,Self-emp-not-inc,119344,10th,6,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,36,United-States,<=50K
10454,41,Private,340148,Some-college,10,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,45,United-States,<=50K
18106,31,State-gov,124020,Assoc-acdm,12,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,>50K
4747,42,Private,107762,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
23553,53,Private,229465,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,55,United-States,>50K


In [37]:
# sample with replacement
df.sample(frac=0.1, replace=True).head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
30314,59,Self-emp-inc,349910,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
1034,76,?,431192,7th-8th,4,Widowed,?,Not-in-family,White,Male,0,0,2,United-States,<=50K
4596,39,Self-emp-not-inc,247975,Some-college,10,Never-married,Craft-repair,Not-in-family,Asian-Pac-Islander,Male,0,0,30,United-States,<=50K
7524,49,Private,225124,HS-grad,9,Widowed,Machine-op-inspct,Unmarried,Black,Female,0,0,40,United-States,<=50K
12309,22,Private,218215,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,40,United-States,<=50K


In [38]:
# sample columns
df.sample(n=2, axis=1).head()

,sex,salary
0,Male,<=50K
1,Male,<=50K
2,Male,<=50K
3,Male,<=50K
4,Female,<=50K


# Pre Processing

### Missing value analysis

In [39]:
# make a copy
preprocessed = df.copy()

In [40]:
# check how many missing values in each column
preprocessed.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
                 ..
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
Length: 15, dtype: int64

In [41]:
# generate some missing values 

# a function to return NaN with 10% probability
def rna(x):
    if np.random.random() > 0.9:
        return np.NaN
    else:
        return x

preprocessed["capital-gain"] = preprocessed["capital-gain"].map(rna)
preprocessed["education-num"] = preprocessed["education-num"].map(rna)

In [42]:
preprocessed[["capital-gain", "education-num", "sex"]].isnull().sum()

capital-gain     3260
education-num    3206
sex                 0
dtype: int64

In [43]:
# impute missing values with either mean, mode or median
val1 = preprocessed["capital-gain"].mean()
val2 = preprocessed["education-num"].mode()

preprocessed["capital-gain"].fillna(val1, inplace=True)
preprocessed["education-num"].fillna(val2, inplace=True)

preprocessed[["capital-gain", "education-num"]].isnull().sum()

capital-gain        0
education-num    3205
dtype: int64

### Dropping columns

In [44]:
cols_to_drop = ["marital-status", "workclass", "native-country", "relationship", "race"]
preprocessed.drop(cols_to_drop, axis=1, inplace=True)

### Encoding catogorical variables

In [45]:
# label encoding

fields = ["education", "occupation", "salary"]
for field in fields:
    le = LabelEncoder()
    preprocessed[field] = le.fit_transform(df[field])

# encoded variable
preprocessed["occupation"].value_counts()

10    4140
3     4099
4     4066
1     3770
12    3650
      ... 
5      994
13     928
11     649
9      149
2        9
Name: occupation, Length: 15, dtype: int64

In [46]:
# one hot encoding

onehot_encode_cols = ["sex"]

for field in onehot_encode_cols:
    # make dummy variables for the data column
    dum = pd.get_dummies(preprocessed[field])

    # trim column names of any unwanted characters
    #dum.columns = np.array([i.strip() for i in dum.columns])
    dum.columns = dum.columns.str.strip().str.lower().str.replace(' ', '_')
    
    # add column name as a prefix
    dum = dum.add_prefix(field + "_")

    # delete column and add one hot encoded columns
    del preprocessed[field]
    preprocessed = pd.concat([dum, preprocessed], axis=1)

preprocessed.head()

,sex_female,sex_male,age,fnlwgt,education,education-num,occupation,capital-gain,capital-loss,hours-per-week,salary
0,0,1,39,77516,9,9.0,1,2174.000000,0,40,0
1,0,1,50,83311,9,13.0,4,0.000000,0,13,0
2,0,1,38,215646,11,9.0,6,1075.442852,0,40,0
3,0,1,53,234721,1,7.0,6,0.000000,0,40,0
4,1,0,28,338409,9,13.0,10,0.000000,0,40,0


### Scale numeric columns

In [47]:
sc = MinMaxScaler()
sc.fit_transform(preprocessed["fnlwgt"].values.reshape(-1, 1))

array([[0.0443019 ],
       [0.0482376 ],
       [0.13811345],
       ...,
       [0.09482688],
       [0.12849934],
       [0.18720338]])

# Selection and Subsetting

In [48]:
# select dataframe rows where column = a specific value
# this is analogus to sql query 'select * from df where age = 39'
df.loc[df.age == 39].head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
28,39,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,80,United-States,<=50K
129,39,Private,365739,Some-college,10,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K
166,39,Federal-gov,235485,Assoc-acdm,12,Never-married,Exec-managerial,Not-in-family,White,Male,0,0,42,United-States,<=50K
297,39,?,157443,Masters,14,Married-civ-spouse,?,Wife,Asian-Pac-Islander,Female,3464,0,40,?,<=50K


In [49]:
# select last column of the dataframe
df.iloc[:,-1].head()

0     <=50K
1     <=50K
2     <=50K
3     <=50K
4     <=50K
Name: salary, dtype: object

In [50]:
fruits = pd.DataFrame({'fruit1': np.random.choice(['apple', 'orange', 'banana'], 10),
                    'fruit2': np.random.choice(['apple', 'orange', 'banana'], 10)})
fruits

,fruit1,fruit2
0,banana,banana
1,orange,orange
2,banana,orange
3,banana,orange
4,apple,banana
5,apple,apple
6,apple,orange
7,banana,orange
8,orange,apple
9,apple,banana


In [51]:
# using where
np.where(fruits.fruit1 == fruits.fruit2)

(array([0, 1, 5], dtype=int64),)

# Change data type of column

In [52]:
# change column data type
df.fnlwgt.astype(np.int64).head()

0     77516
1     83311
2    215646
3    234721
4    338409
Name: fnlwgt, dtype: int64

# Merging and Concatinating 
  
Merge - merge 2 dataframes using different kinds of joins.  
Concat - concatinate dataframes along index or columns

In [53]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [54]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [55]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_n = pd.DataFrame(raw_data, columns = ['subject_id','test_id'])
df_n

,subject_id,test_id
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


In [56]:
# join two dataframes
# axis = 1 join along columns, axis = 0 join along index
pd.concat([df_a, df_b], axis = 0)

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [57]:
# when joining along index, might wanna create new index
# to ignore existing indexes and make new index
pd.concat([df_a, df_b], axis = 0, ignore_index=True)

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
5,4,Billy,Bonder
6,5,Brian,Black
7,6,Bran,Balwner
8,7,Bryce,Brice
9,8,Betty,Btisan


In [58]:
# inner join
pd.merge(df_a, df_n, on='subject_id')

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16


In [59]:
# right join
pd.merge(df_a, df_n, on='subject_id', how="outer")

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16
5,7,NaN,NaN,14
6,8,NaN,NaN,15
7,9,NaN,NaN,1
8,10,NaN,NaN,61
9,11,NaN,NaN,16


In [60]:
pd.merge(df_a, df_n, on='subject_id', how="left")

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16


In [61]:
pd.merge(df_a, df_n, on='subject_id', how="right")

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16
5,7,NaN,NaN,14
6,8,NaN,NaN,15
7,9,NaN,NaN,1
8,10,NaN,NaN,61
9,11,NaN,NaN,16


In [62]:
df1 = pd.DataFrame({'fruit': ['apple', 'banana', 'orange'] * 3,
                    'weight': ['high', 'medium', 'low'] * 3,
                    'price': np.random.randint(0, 15, 9)})

df2 = pd.DataFrame({'pazham': ['apple', 'orange', 'pine'] * 2,
                    'kilo': ['high', 'low'] * 3,
                    'price': np.random.randint(0, 15, 6)})

# Solution
pd.merge(df1, df2, how='inner', left_on=['fruit', 'weight'], right_on=['pazham', 'kilo'], suffixes=['_left', '_right'])

,fruit,weight,price_left,pazham,kilo,price_right
0,apple,high,1,apple,high,9
1,apple,high,1,apple,high,9
2,apple,high,11,apple,high,9
3,orange,low,13,orange,low,0
4,orange,low,8,orange,low,0
5,orange,low,4,orange,low,0


# Lag and Lead

In [63]:
tmp = pd.DataFrame(np.random.randint(1, 100, 20).reshape(-1, 4), columns = list('abcd'))

# use shift
tmp['a_lag1'] = tmp['a'].shift(1)
tmp['b_lead1'] = tmp['b'].shift(-1)
print(tmp)

    a   b   c   d  a_lag1  b_lead1
0  25   3  92  14     NaN     42.0
1   5  42  47   9    25.0     14.0
2  95  14  93  92     5.0     61.0
3  36  61  43  60    95.0     86.0
4  96  86  27  95    36.0      NaN


# Datetime functions 

In [64]:
# create date ranges
pd.date_range('2000', freq='10YS', periods=5)

DatetimeIndex(['2000-01-01', '2010-01-01', '2020-01-01', '2030-01-01',
               '2040-01-01'],
              dtype='datetime64[ns]', freq='10AS-JAN')

In [65]:
# convert string to datetime
pd.to_datetime('2019-01-01', format='%Y-%m-%d')

Timestamp('2019-01-01 00:00:00')

In [66]:
# get weekday 
# mode formats for strftime - https://strftime.org/
import datetime
day = pd.to_datetime('2019-01-01', format='%Y-%m-%d')
datetime.datetime.strftime(day, '%A')

'Tuesday'

In [67]:
# can also use day_name to get weekday
day.day_name()

'Tuesday'

In [68]:
# timedelta 
day + pd.Timedelta(days=1)

Timestamp('2019-01-02 00:00:00')

In [69]:
# date offsets
pd.offsets.BDay()

<BusinessDay>

In [70]:
# going forward 2 business days
day = pd.Timestamp('2018-01-05')
two_business_days = 2 * pd.offsets.BDay()
two_business_days.apply(day)

Timestamp('2018-01-09 00:00:00')

In [71]:
day.day_name(), two_business_days.apply(day).day_name()

('Friday', 'Tuesday')

In [72]:
# alternate syntax
day, day + pd.offsets.Week()

(Timestamp('2018-01-05 00:00:00'), Timestamp('2018-01-12 00:00:00'))

In [73]:
# using DateOffset
pd.DateOffset(months=2)

<DateOffset: months=2>

In [74]:
day + pd.DateOffset(months=2)

Timestamp('2018-03-05 00:00:00')

# Timeseries

In [75]:
url = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv"
crime = pd.read_csv(url)
crime.head()

,Year,Population,Total,Violent,Property,Murder,Forcible_Rape,Robbery,Aggravated_assault,Burglary,Larceny_Theft,Vehicle_Theft
0,1960,179323175,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1,1961,182992000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
2,1962,185771000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
3,1963,188483000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
4,1964,191141000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


In [76]:
# convert to datetime and set index
crime.Year = pd.to_datetime(crime.Year, format='%Y')
crime = crime.set_index('Year', drop = True)
crime.head()

,Population,Total,Violent,Property,Murder,Forcible_Rape,Robbery,Aggravated_assault,Burglary,Larceny_Theft,Vehicle_Theft
Year,,,,,,,,,,,
1960-01-01,179323175,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1961-01-01,182992000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
1962-01-01,185771000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
1963-01-01,188483000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
1964-01-01,191141000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


In [77]:
# Group the year by decades and sum the values

# To learn more about .resample (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html)
# To learn more about Offset Aliases (http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)

# Uses resample to sum each decade
crimes = crime.resample('10AS').sum()

# Uses resample to get the max value only for the "Population" column
population = crime['Population'].resample('10AS').max()

# Updating the "Population" column
crimes['Population'] = population

crimes

,Population,Total,Violent,Property,Murder,Forcible_Rape,Robbery,Aggravated_assault,Burglary,Larceny_Theft,Vehicle_Theft
Year,,,,,,,,,,,
1960-01-01,201385000,49295900,4134930,45160900,106180,236720,1633510,2158520,13321100,26547700,5292100
1970-01-01,220099000,100991600,9607930,91383800,192230,554570,4159020,4702120,28486000,53157800,9739900
1980-01-01,248239000,131123369,14074328,117048900,206439,865639,5383109,7619130,33073494,72040253,11935411
1990-01-01,272690813,136582146,17527048,119053499,211664,998827,5748930,10568963,26750015,77679366,14624418
2000-01-01,307006550,115012044,13968056,100944369,163068,922499,4230366,8652124,21565176,67970291,11412834
2010-01-01,318857056,50167967,6072017,44095950,72867,421059,1749809,3764142,10125170,30401698,3569080


# Trick questions

In [78]:
tmp = pd.DataFrame(np.arange(25).reshape(5, -1), columns=list('abcde'))
tmp

,a,b,c,d,e
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19
4,20,21,22,23,24


In [79]:
# swap rows of dataframe
def swap_rows(df, i1, i2):
    a, b = df.iloc[i1, :].copy(), df.iloc[i2, :].copy()
    df.iloc[i1, :], df.iloc[i2, :] = b, a
    return df

swap_rows(tmp, 1, 2)

,a,b,c,d,e
0,0,1,2,3,4
1,10,11,12,13,14
2,5,6,7,8,9
3,15,16,17,18,19
4,20,21,22,23,24


In [80]:
# reverse all rows in dataframe
tmp.iloc[::-1, :]

,a,b,c,d,e
4,20,21,22,23,24
3,15,16,17,18,19
2,5,6,7,8,9
1,10,11,12,13,14
0,0,1,2,3,4


In [81]:
# format dataframe column into percentile
tmp = pd.DataFrame(np.random.random(4), columns=['random'])
tmp.random.apply(lambda x: "{:.2%}".format(x))

0    31.26%
1    27.70%
2    88.72%
3    98.04%
Name: random, dtype: object

In [82]:
# remove scientific notation
tmp = pd.DataFrame(np.random.random(4)**10, columns=['random'])
tmp

,random
0,0.005146
1,0.000088
2,0.001517
3,0.022430


In [83]:
tmp.applymap(lambda x: '%.4f' % x)

,random
0,0.0051
1,0.0001
2,0.0015
3,0.0224


In [90]:

state = np.random.RandomState(100)
ser = pd.Series(state.normal(10, 5, 25))
ser

0      1.251173
1     11.713402
2     15.765179
3      8.737820
4     14.906604
        ...    
20    18.094908
21    17.708026
22     8.740604
23     5.787821
24    10.922593
Length: 25, dtype: float64

In [91]:
# get percentiles
np.percentile(ser, q=[0, 25, 50, 75, 100])

array([ 1.25117263,  7.70986507, 10.92259345, 13.36360403, 18.0949083 ])